## 基础OpenAI Tools Agent

In [4]:
from langchain_ollama import ChatOllama
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
import requests

# 定义工具
@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息"""
    # 模拟天气API调用
    weather_data = {
        "北京": "晴天，25°C",
        "上海": "多云，22°C",
        "广州": "雨天，28°C"
    }
    return weather_data.get(city, f"{city}的天气信息暂不可用")

@tool
def calculate(expression: str) -> str:
    """计算数学表达式"""
    try:
        result = eval(expression)
        return f"{expression} = {result}"
    except:
        return "计算错误，请检查表达式"

# 创建模型
llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:4b",
    temperature=0
)

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的AI助手，可以使用提供的工具来回答问题。"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 工具列表
tools = [get_weather, calculate]

# 创建agent
agent = create_openai_tools_agent(llm, tools, prompt)

# 创建agent执行器
agent_executor = AgentExecutor(agent=agent, tools=tools
                               # , verbose=True
                               )

# 测试
if __name__ == "__main__":
    # 测试天气查询
    result1 = agent_executor.invoke({"input": "北京今天天气怎么样？"})
    print("天气查询结果:", result1["output"])

    # 测试计算
    result2 = agent_executor.invoke({"input": "计算 25 * 4 + 10"})
    print("计算结果:", result2["output"])

    # 测试复合任务
    result3 = agent_executor.invoke({"input": "如果北京今天的温度是25度，转换成华氏度是多少？"})
    print("复合任务结果:", result3["output"])

天气查询结果: <think>
好的，用户问北京今天的天气，我调用了get_weather函数，返回的是晴天，25°C。现在需要把结果用自然的中文回复给用户。首先，确认信息是否正确，北京今天确实晴天，温度25度。然后组织语言，保持口语化，避免使用专业术语。可以这样回答：“北京今天天气晴朗，温度25°C，适合外出活动。” 检查是否需要补充其他信息，比如建议用户注意防晒或者穿衣，但原数据没有提供更多细节，所以保持简洁。确保回复准确且友好。
</think>

北京今天天气晴朗，温度25°C，适合外出活动。
计算结果: <think>
好的，用户让我计算25乘以4再加上10的结果。首先，我需要确认用户的需求是否明确。表达式看起来没问题，先乘后加，顺序正确。接下来，我需要调用计算函数，参数是表达式字符串。确保参数正确传递，没有遗漏或错误的符号。然后，等待函数返回结果，再将结果以清晰的方式回复给用户。结果应该是110，所以直接告诉用户答案即可。检查是否有其他可能的疑问，比如单位或是否需要进一步解释，但用户的问题很简单，可能不需要额外说明。确认回复正确后，发送给用户。
</think>

计算结果为：110
复合任务结果: <think>
好的，用户问的是北京今天的温度25度转换成华氏度的结果。我需要先确认转换公式是否正确。摄氏度转华氏度的公式是°F = °C × 9/5 + 32，所以25代入的话就是25×9/5=45，加上32等于77。之前调用calculate函数计算的结果也是77.0，说明计算是对的。用户可能是在旅行或者需要不同单位的温度参考，所以给出准确的转换结果很重要。确保回答清晰，直接给出华氏度数值即可。
</think>

25度摄氏度转换成华氏度是77度。


## 示例2：Tool Calling Agent（支持多种模型）

In [3]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from typing import List, Dict
import json

# 定义更复杂的工具
@tool
def search_database(query: str) -> str:
    """在数据库中搜索信息"""
    # 模拟数据库搜索
    database = {
        "python": "Python是一种高级编程语言",
        "langchain": "LangChain是一个用于构建LLM应用的框架",
        "ai": "人工智能是计算机科学的一个分支"
    }

    for key, value in database.items():
        if key.lower() in query.lower():
            return f"找到相关信息: {value}"
    return "未找到相关信息"

@tool
def file_operations(operation: str, filename: str, content: str = "") -> str:
    """执行文件操作（读取、写入、删除）"""
    try:
        if operation == "write":
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(content)
            return f"成功写入文件 {filename}"
        elif operation == "read":
            with open(filename, 'r', encoding='utf-8') as f:
                content = f.read()
            return f"文件内容: {content}"
        elif operation == "delete":
            import os
            os.remove(filename)
            return f"成功删除文件 {filename}"
        else:
            return "不支持的操作类型"
    except Exception as e:
        return f"文件操作失败: {str(e)}"

@tool
def data_analyzer(data: str) -> str:
    """分析数据并返回统计信息"""
    try:
        # 假设输入是逗号分隔的数字
        numbers = [float(x.strip()) for x in data.split(',')]

        stats = {
            "总数": len(numbers),
            "平均值": sum(numbers) / len(numbers),
            "最大值": max(numbers),
            "最小值": min(numbers),
            "总和": sum(numbers)
        }

        return f"数据分析结果: {json.dumps(stats, ensure_ascii=False, indent=2)}"
    except Exception as e:
        return f"数据分析失败: {str(e)}"

# 创建工具列表
tools = [search_database, file_operations, data_analyzer]

# 初始化本地模型（使用 Ollama）
llm = ChatOllama(
    model="qwen3:0.6b",
    temperature=0.1
)

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个智能助手，可以使用多种工具来帮助用户。

可用工具:
- search_database: 搜索数据库信息
- file_operations: 执行文件操作
- data_analyzer: 分析数据

请根据用户需求选择合适的工具，并提供详细的回答。"""),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 创建代理
agent = create_tool_calling_agent(llm=llm, tools=tools, prompt=prompt)

# 创建代理执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,
    early_stopping_method="generate"
)

def test_tool_calling_agent():
    test_cases = [
        "搜索关于 Python 的信息",
        "创建一个名为 test.txt 的文件，内容是 'Hello, LangChain!'",
        "分析这些数据: 10, 20, 30, 40, 50",
        "读取刚才创建的 test.txt 文件",
        "先搜索 AI 相关信息，然后将结果保存到 ai_info.txt 文件中"
    ]

    for i, query in enumerate(test_cases, 1):
        print(f"\n=== 测试 {i} ===")
        print(f"用户问题: {query}")
        print("-" * 60)

        try:
            result = agent_executor.invoke({"input": query})
            print(f"代理回答: {result['output']}")
        except Exception as e:
            print(f"执行出错: {str(e)}")

if __name__ == "__main__":
    test_tool_calling_agent()


=== 测试 1 ===
用户问题: 搜索关于 Python 的信息
------------------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `search_database` with `{'query': 'Python'}`
responded: <think>
好的，用户想搜索关于Python的信息。我需要看看可用的工具中有没有能处理这个请求的。提供的工具有search_database、file_operations和data_analyzer。首先，用户的问题是关于Python的，可能需要查找资料或者数据。search_database的描述是搜索数据库信息，而data_analyzer是分析数据并返回统计信息。file_operations是文件操作，但用户的问题没有提到文件相关的内容。所以最合适的应该是search_database来搜索Python的信息。接下来，我需要构造一个调用search_database的JSON对象，参数是query，值为“Python”。这样就能返回关于Python的相关信息了。
</think>



找到相关信息: Python是一种高级编程语言<think>
好的，用户之前询问了关于Python的信息，我通过调用search_database工具找到了相关信息。现在需要根据用户最新的回复来生成最终回答。用户现在给出的信息是“找到相关信息: Python是一种高级编程语言”，我需要确认是否需要进一步处理或补充。用户可能希望了解更多信息，比如Python的用途或相关知识。不过根据之前的工具使用情况，用户可能已经获取了足够的信息，因此直接总结即可。不需要调用其他工具，因为用户的问题已经得到回答。保持简洁，确保用户理解信息。
</think>

Python是一种高级编程语言，广泛用于数据处理、算法开发和科学计算等领域。它语法简洁、易读性强，适合初学者学习，同时拥有强大的库支持，如NumPy、Pandas等，使得开发者能够高效完成复杂任务。

> Finished chain.
代理回答: <think>
好的，用户之前询问了关于Python的信息，我通过调用search

## 示例3：ReAct Agent（推理-行动循环）

In [4]:
from langchain.agents import AgentExecutor, create_react_agent
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain import hub
import requests
import json
from datetime import datetime

# 定义 ReAct 风格的工具
@tool
def web_search(query: str) -> str:
    """模拟网络搜索功能"""
    # 这里模拟搜索结果
    search_results = {
        "天气": "今天天气晴朗，温度适宜",
        "新闻": "最新科技新闻：AI技术持续发展",
        "股票": "股市今日表现平稳",
        "python": "Python 3.12 发布了新特性"
    }

    for key, value in search_results.items():
        if key in query.lower():
            return f"搜索结果: {value}"

    return f"关于 '{query}' 的搜索结果: 找到相关信息，建议进一步查询具体内容"

@tool
def time_tool() -> str:
    """获取当前时间"""
    now = datetime.now()
    return f"当前时间: {now.strftime('%Y-%m-%d %H:%M:%S')}"

@tool
def math_calculator(expression: str) -> str:
    """执行数学计算"""
    try:
        # 安全的数学表达式计算
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "错误: 包含不允许的字符"

        result = eval(expression)
        return f"计算结果: {expression} = {result}"
    except Exception as e:
        return f"计算错误: {str(e)}"

@tool
def text_processor(text: str, operation: str) -> str:
    """处理文本（转换大小写、计算长度等）"""
    operations = {
        "upper": text.upper(),
        "lower": text.lower(),
        "length": f"文本长度: {len(text)} 字符",
        "words": f"单词数量: {len(text.split())}",
        "reverse": text[::-1]
    }

    if operation in operations:
        return f"文本处理结果: {operations[operation]}"
    else:
        return f"支持的操作: {', '.join(operations.keys())}"

# 创建工具列表
tools = [web_search, time_tool, math_calculator, text_processor]

# 初始化模型
llm = ChatOllama(
    model="qwen3:0.6b",
    temperature=0
)

# 获取 ReAct 提示模板
react_prompt = hub.pull("hwchase17/react")

# 创建 ReAct 代理
agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)

# 创建代理执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=10,
    handle_parsing_errors=True
)

def test_react_agent():
    test_scenarios = [
        "现在几点了？然后计算 15 * 8 的结果",
        "搜索关于 Python 的信息，并告诉我 'Hello World' 这个文本有多少个字符",
        "计算 (100 + 50) / 3，然后将结果转换为大写文本格式",
        "获取当前时间，搜索今天的天气，并计算 24 * 60 * 60（一天的秒数）"
    ]

    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n{'='*20} 场景 {i} {'='*20}")
        print(f"任务: {scenario}")
        print("-" * 60)

        try:
            result = agent_executor.invoke({"input": scenario})
            print(f"\n最终结果: {result['output']}")
        except Exception as e:
            print(f"执行出错: {str(e)}")

        print("\n" + "="*60)

if __name__ == "__main__":
    test_react_agent()

D:\javaDevelopment\program_project\Pycharm_project\LangChainNoteBook\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(



==================== 场景 1 ====================
任务: 现在几点了？然后计算 15 * 8 的结果
------------------------------------------------------------


> Entering new AgentExecutor chain...
<think>
好的，我现在需要回答用户的问题。用户的问题有两个部分：首先问现在几点了，然后计算15乘以8的结果。我需要先确定当前的时间，然后进行数学计算。

首先，用户的问题看起来是两个独立的问题，所以可能需要分开处理。不过用户的问题可能希望我同时回答这两个部分。不过根据问题的结构，可能需要先回答当前时间，然后计算乘法。不过用户的问题可能希望我先确认时间，再计算结果。不过用户的问题可能只是两个问题，所以可能需要分开回答。

不过根据用户的问题，可能需要先确认时间，然后计算乘法。所以步骤应该是：

1. 确定当前时间。
2. 计算15乘以8的结果。

现在，我需要先确认当前时间。假设当前时间是2023年10月15日，比如用户的问题可能是在某个时间点。不过实际情况下，我需要使用网络搜索工具来获取当前时间。所以第一步是使用web_search工具搜索当前时间，然后得到当前时间，再计算乘法。

例如，假设现在的时间是2023年10月15日，那么第一步是搜索当前时间，得到“2023-10-15”，然后计算15*8=120。所以最终答案应该是“现在几点了？120”。
</think>

Thought: 我需要先确认当前时间，然后计算15乘以8的结果。  
Action: web_search  
Action Input: "现在几点了？然后计算 15 * 8 的结果"  关于 '现在几点了？然后计算 15 * 8 的结果' 的搜索结果: 找到相关信息，建议进一步查询具体内容<think>
好的，现在我需要回答用户的问题。首先，用户问现在几点了，然后计算15乘8的结果。我需要先确认当前时间，然后进行数学计算。

第一步，确认当前时间。假设当前时间是2023年10月15日，但实际时间可能不同。我需要使用网络搜索工具来获取当前时间。搜索结果可能显示当前时间，比如“2023-10-15”。然后计算15乘8，得到120。所以最终答案应该是

## 示例4：自定义 Agent

In [9]:
print("示例4：自定义 Agent")

示例4：自定义 Agent


## 示例5：多代理协作系统

In [6]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from typing import Dict, List, Any
import json
from datetime import datetime

# 专业化工具定义
@tool
def research_tool(topic: str) -> str:
    """研究工具 - 收集信息"""
    research_db = {
        "ai": "人工智能是模拟人类智能的技术，包括机器学习、深度学习等",
        "blockchain": "区块链是分布式账本技术，具有去中心化、不可篡改等特点",
        "quantum": "量子计算利用量子力学原理进行计算，具有巨大的计算潜力",
        "climate": "气候变化是全球性挑战，需要采取紧急行动减少温室气体排放"
    }

    for key, value in research_db.items():
        if key in topic.lower():
            return f"研究结果: {value}"

    return f"关于 '{topic}' 的研究: 这是一个值得深入探讨的话题，建议进一步调研"

@tool
def analysis_tool(data: str) -> str:
    """分析工具 - 数据分析"""
    try:
        # 模拟数据分析
        if "," in data:
            numbers = [float(x.strip()) for x in data.split(",")]
            analysis = {
                "数据点数": len(numbers),
                "平均值": round(sum(numbers) / len(numbers), 2),
                "最大值": max(numbers),
                "最小值": min(numbers),
                "趋势": "上升" if numbers[-1] > numbers[0] else "下降"
            }
            return f"数据分析结果: {json.dumps(analysis, ensure_ascii=False)}"
        else:
            return f"文本分析: 长度 {len(data)} 字符，包含 {len(data.split())} 个词"
    except Exception as e:
        return f"分析失败: {str(e)}"

@tool
def writing_tool(content: str, style: str = "formal") -> str:
    """写作工具 - 内容创作"""
    styles = {
        "formal": "正式的学术风格",
        "casual": "轻松的对话风格",
        "technical": "技术文档风格",
        "creative": "创意写作风格"
    }

    if style in styles:
        return f"已按照{styles[style]}重写内容: {content}"
    else:
        return f"内容创作完成: {content} (使用默认风格)"

@tool
def review_tool(content: str) -> str:
    """审查工具 - 质量检查"""
    issues = []

    if len(content) < 10:
        issues.append("内容过短")
    if not any(char.isupper() for char in content):
        issues.append("缺少大写字母")
    if content.count(".") == 0:
        issues.append("缺少句号")

    if issues:
        return f"审查发现问题: {', '.join(issues)}"
    else:
        return "内容审查通过，质量良好"

# 创建专业化代理
class SpecializedAgent:
    def __init__(self, name: str, role: str, tools: List, llm):
        self.name = name
        self.role = role
        self.tools = tools
        self.llm = llm

        # 创建专业化提示
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", f"""你是 {name}，专门负责 {role}。

你的工具: {[tool.name for tool in tools]}

请专注于你的专业领域，提供高质量的服务。
如果任务超出你的专业范围，请明确说明并建议转交给其他专家。"""),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
            ("placeholder", "{agent_scratchpad}")
        ])

        # 创建代理
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.tools,
            prompt=self.prompt
        )

        # 创建执行器
        self.executor = AgentExecutor(
            agent=self.agent,
            tools=self.tools,
            verbose=True,
            max_iterations=3
        )

    def process(self, task: str) -> str:
        """处理任务"""
        try:
            result = self.executor.invoke({"input": task})
            return result["output"]
        except Exception as e:
            return f"{self.name} 处理失败: {str(e)}"

# 多代理协调器
class MultiAgentCoordinator:
    def __init__(self):
        self.llm = ChatOllama(model="qwen3:0.6b", temperature=0.1)

        # 创建专业化代理
        self.agents = {
            "researcher": SpecializedAgent(
                "研究员", "信息收集和研究", [research_tool], self.llm
            ),
            "analyst": SpecializedAgent(
                "分析师", "数据分析和洞察", [analysis_tool], self.llm
            ),
            "writer": SpecializedAgent(
                "写作专家", "内容创作和编辑", [writing_tool], self.llm
            ),
            "reviewer": SpecializedAgent(
                "审查员", "质量控制和审查", [review_tool], self.llm
            )
        }

        self.workflow_history = []

    def route_task(self, task: str) -> str:
        """任务路由 - 决定哪个代理处理任务"""
        routing_keywords = {
            "researcher": ["研究", "调查", "信息", "资料", "查找"],
            "analyst": ["分析", "数据", "统计", "计算", "趋势"],
            "writer": ["写作", "创作", "编辑", "文章", "内容"],
            "reviewer": ["审查", "检查", "质量", "评估", "验证"]
        }

        task_lower = task.lower()
        for agent_name, keywords in routing_keywords.items():
            if any(keyword in task_lower for keyword in keywords):
                return agent_name

        return "researcher"  # 默认路由到研究员

    def execute_workflow(self, task: str) -> Dict[str, Any]:
        """执行完整工作流"""
        print(f"\n开始执行任务: {task}")
        print("="*60)

        workflow_result = {
            "original_task": task,
            "steps": [],
            "final_result": "",
            "timestamp": datetime.now().isoformat()
        }

        # 步骤1: 研究
        print("\n步骤1: 信息研究")
        research_result = self.agents["researcher"].process(f"研究任务: {task}")
        workflow_result["steps"].append({"step": "research", "result": research_result})

        # 步骤2: 分析
        print("\n步骤2: 数据分析")
        analysis_task = f"分析研究结果: {research_result}"
        analysis_result = self.agents["analyst"].process(analysis_task)
        workflow_result["steps"].append({"step": "analysis", "result": analysis_result})

        # 步骤3: 写作
        print("\n步骤3: 内容创作")
        writing_task = f"基于研究和分析结果创作内容: 研究结果: {research_result} 分析结果: {analysis_result}"
        writing_result = self.agents["writer"].process(writing_task)
        workflow_result["steps"].append({"step": "writing", "result": writing_result})

        # 步骤4: 审查
        print("\n步骤4: 质量审查")
        review_task = f"审查最终内容: {writing_result}"
        review_result = self.agents["reviewer"].process(review_task)
        workflow_result["steps"].append({"step": "review", "result": review_result})

        # 整合最终结果
        workflow_result["final_result"] = f"""
任务完成报告:

原始任务: {task}

研究结果: {research_result}

分析结果: {analysis_result}

创作内容: {writing_result}

质量审查: {review_result}

完成时间: {workflow_result["timestamp"]}
"""

        self.workflow_history.append(workflow_result)
        return workflow_result

def test_multi_agent_system():
    coordinator = MultiAgentCoordinator()

    test_tasks = [
        "研究人工智能的发展趋势并写一份报告",
        "分析数据 10,15,20,25,30,35 并创作一份分析报告",
        "调研区块链技术并写一篇技术文章"
    ]

    for i, task in enumerate(test_tasks, 1):
        print(f"\n{'='*20} 任务 {i} {'='*20}")
        result = coordinator.execute_workflow(task)
        print(f"\n最终结果:\n{result['final_result']}")
        print("\n" + "="*80)

if __name__ == "__main__":
    test_multi_agent_system()


==================== 任务 1 ====================

开始执行任务: 研究人工智能的发展趋势并写一份报告

步骤1: 信息研究


> Entering new AgentExecutor chain...

Invoking: `research_tool` with `{'topic': '人工智能的发展趋势'}`
responded: <think>
好的，用户让我研究人工智能的发展趋势并写一份报告。首先，我需要确认用户的需求是什么。他们可能希望了解当前AI的未来方向，比如技术应用、伦理问题或者政策影响。接下来，我需要考虑使用提供的工具。用户提供的工具只有一个research_tool，用于收集信息。所以，我应该调用这个工具，指定主题为人工智能的发展趋势。然后，根据工具的参数，构造一个包含topic参数的JSON对象，确保主题明确。最后，用正确的XML格式返回工具调用。
</think>



关于 '人工智能的发展趋势' 的研究: 这是一个值得深入探讨的话题，建议进一步调研
Invoking: `research_tool` with `{'topic': '人工智能的发展趋势'}`
responded: <think>
好的，用户现在需要关于人工智能发展趋势的研究，并要求写一份报告。我需要先确认用户的需求是否已经满足。用户之前已经请求过研究工具，现在可能希望得到更详细的分析。我需要确保在回复中明确说明当前的研究状态，并提供进一步的信息。同时，保持专业性和清晰的结构，确保用户能够顺利获取所需内容。最后，检查是否有遗漏的步骤，确保回复准确无误。
</think>



关于 '人工智能的发展趋势' 的研究: 这是一个值得深入探讨的话题，建议进一步调研<think>
好的，用户现在需要关于人工智能发展趋势的研究，并要求写一份报告。我需要确认用户的需求是否已经满足。用户可能希望得到更详细的分析，因此需要进一步明确研究方向。同时，保持专业性和清晰的结构，确保用户能够顺利获取所需内容。最后，检查是否有遗漏的步骤，确保回复准确无误。

> Finished chain.

步骤2: 数据分析


> Entering new AgentExecutor chain...
<think>
好的，用户需要分析人工智能发展趋势并撰写

KeyboardInterrupt: 

## else


In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.runnables import RunnableSequence
from langchain.agents import AgentExecutor, AgentStep
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.openai.output_parser import OpenAIFunctionsAgentOutputParser
from typing import List

# 定义更多工具
@tool
def search_database(query: str) -> str:
    """搜索数据库中的信息"""
    # 模拟数据库搜索
    database = {
        "python": "Python是一种高级编程语言",
        "langchain": "LangChain是一个用于构建LLM应用的框架",
        "ai": "人工智能是计算机科学的一个分支"
    }
    for key, value in database.items():
        if key in query.lower():
            return value
    return "未找到相关信息"

@tool
def get_current_time() -> str:
    """获取当前时间"""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@tool
def translate_text(text: str, target_language: str = "英文") -> str:
    """翻译文本到指定语言"""
    # 简单的翻译模拟
    translations = {
        "你好": {"英文": "Hello", "日文": "こんにちは"},
        "谢谢": {"英文": "Thank you", "日文": "ありがとう"},
        "再见": {"英文": "Goodbye", "日文": "さようなら"}
    }

    if text in translations and target_language in translations[text]:
        return translations[text][target_language]
    return f"无法翻译'{text}'到{target_language}"

# 创建模型并绑定工具
llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen2.5:3b",
    temperature=0
)

tools = [search_database, get_current_time, translate_text]

# 将工具转换为函数格式
llm_with_tools = llm.bind_tools(tools)

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一个智能助手，可以使用以下工具：
    - search_database: 搜索数据库信息
    - get_current_time: 获取当前时间
    - translate_text: 翻译文本

    请根据用户问题选择合适的工具来回答。"""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 创建自定义agent
def create_custom_agent():
    agent = RunnableSequence.from([
        {
            "input": lambda x: x["input"],
            "agent_scratchpad": lambda x: format_to_openai_function_messages(x["steps"])
        },
        prompt,
        llm_with_tools,
        OpenAIFunctionsAgentOutputParser()
    ])
    return agent

# 创建并运行agent
if __name__ == "__main__":
    agent = create_custom_agent()
    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        verbose=True
    )

    # 测试各种功能
    test_queries = [
        "现在几点了？",
        "搜索一下Python的信息",
        "把'你好'翻译成英文",
        "搜索LangChain并告诉我现在的时间"
    ]

    for query in test_queries:
        print(f"\n查询: {query}")
        result = agent_executor.invoke({"input": query})
        print(f"回答: {result['output']}")
        print("-" * 50)

In [ ]:
# 创建模型
llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:4b",
    temperature=0
)

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个有用的AI助手，可以使用提供的工具来回答问题。"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# 工具列表
tools = [get_weather, calculate]

# 创建agent
agent = create_openai_tools_agent(llm, tools, prompt)

# 创建agent执行器
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [2]:
"""
基础 OpenAI Tools Agent 示例
"""
from langchain_ollama import ChatOllama
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
import requests
from datetime import datetime

# 定义工具
@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息"""
    weather_data = {
        "北京": "晴天，25°C，湿度60%",
        "上海": "多云，22°C，湿度70%",
        "广州": "雨天，28°C，湿度85%",
        "深圳": "晴天，30°C，湿度55%"
    }
    return weather_data.get(city, f"{city}的天气信息暂不可用")

@tool
def calculate(expression: str) -> str:
    """计算数学表达式，支持基本运算"""
    try:
        # 安全的数学表达式计算
        allowed_chars = set('0123456789+-*/.() ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return f"{expression} = {result}"
        else:
            return "表达式包含不允许的字符"
    except Exception as e:
        return f"计算错误: {str(e)}"

@tool
def get_current_time() -> str:
    """获取当前时间"""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def main():
    # 创建模型
    llm = ChatOllama(
        base_url="http://localhost:11434",
        model="qwen3:0.6b",
        temperature=0
    )

    # 创建提示模板
    prompt = ChatPromptTemplate.from_messages([
        ("system", """你是一个有用的AI助手，可以使用提供的工具来回答问题。
        可用工具：
        - get_weather: 获取城市天气信息
        - calculate: 进行数学计算
        - get_current_time: 获取当前时间

        请根据用户问题选择合适的工具。"""),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ])

    # 工具列表
    tools = [get_weather, calculate, get_current_time]

    # 创建agent
    agent = create_openai_tools_agent(llm, tools, prompt)

    # 创建agent执行器
    agent_executor = AgentExecutor(
        agent=agent,
        tools=tools,
        verbose=True,
        max_iterations=3
    )

    # 测试用例
    test_cases = [
        "北京今天天气怎么样？",
        "计算 25 * 4 + 10",
        "现在几点了？",
        "如果北京今天的温度是25度，转换成华氏度是多少？",
        "上海和广州哪个城市温度更高？"
    ]

    print("🚀 基础 OpenAI Tools Agent 示例")
    print("=" * 60)

    for i, query in enumerate(test_cases, 1):
        print(f"\n📝 测试 {i}: {query}")
        try:
            result = agent_executor.invoke({"input": query})
            print(f"✅ 回答: {result['output']}")
        except Exception as e:
            print(f"❌ 错误: {str(e)}")
        print("-" * 40)

if __name__ == "__main__":
    main()

🚀 基础 OpenAI Tools Agent 示例

📝 测试 1: 北京今天天气怎么样？


> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': '北京'}`
responded: <think>
好的，用户问的是北京今天天气怎么样。我需要使用get_weather工具来获取北京的天气信息。首先，确认工具的参数要求，参数是城市名称，所以应该构造一个{"city": "北京"}的参数。然后，调用get_weather函数，返回结果后，用自然语言回答用户的问题，确保信息准确。
</think>



晴天，25°C，湿度60%<think>
好的，用户现在提供了天气信息，我需要确认回复是否准确。首先，用户的问题是关于北京今天的天气，我调用了get_weather工具，参数是北京，返回了晴天、25°C和60%湿度的信息。接下来，我需要将这些信息用自然的中文组织起来，确保用户能清楚理解当前的天气状况。同时，检查是否有需要进一步询问的地方，比如是否需要建议或者更多细节，但用户的问题已经明确，所以保持简洁即可。确认没有遗漏或错误后，给出最终回复。
</think>

北京今天晴天，温度25°C，湿度60%。需要我为您推荐其他天气情况吗？

> Finished chain.
✅ 回答: <think>
好的，用户现在提供了天气信息，我需要确认回复是否准确。首先，用户的问题是关于北京今天的天气，我调用了get_weather工具，参数是北京，返回了晴天、25°C和60%湿度的信息。接下来，我需要将这些信息用自然的中文组织起来，确保用户能清楚理解当前的天气状况。同时，检查是否有需要进一步询问的地方，比如是否需要建议或者更多细节，但用户的问题已经明确，所以保持简洁即可。确认没有遗漏或错误后，给出最终回复。
</think>

北京今天晴天，温度25°C，湿度60%。需要我为您推荐其他天气情况吗？
----------------------------------------

📝 测试 2: 计算 25 * 4 + 10


> Entering new AgentExecutor chain...

Invoking: `calculate` with `{'express

In [1]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
import datetime

# 1. 准备 LLM 和 Tools
# llm = ChatOpenAI(model="gpt-4o")

llm = ChatOllama(
    model="qwen3:4b",

)

@tool
def get_current_time() -> str:
    """在你需要获取当前时间的时候使用这个工具。"""
    return str(datetime.datetime.now())

tools = [get_current_time]

# 2. 创建 Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# 3. 创建 Agent
agent = create_tool_calling_agent(llm, tools, prompt)

# 4. 创建 AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 5. 运行
agent_executor.invoke({"input": "现在几点了？"})



> Entering new AgentExecutor chain...

Invoking: `get_current_time` with `{}`
responded: <think>
好的，用户问现在几点了，我需要告诉他当前的时间。但问题是我没有内置的时钟功能，所以得用提供的工具来获取当前时间。看看工具里有没有相关的函数，哦，对，有一个get_current_time的函数，不需要参数，直接调用就行。调用之后应该就能得到当前的时间，然后回复用户就可以了。不过得确保正确调用，避免错误。确认一下函数描述，确实是用来获取当前时间的，所以没问题。那现在就调用这个函数，返回给用户当前的时间。
</think>



2025-07-24 18:59:41.573015<think>
好的，用户问现在几点了，我需要调用get_current_time函数来获取当前时间。之前已经调用了这个函数，返回的时间是2025-07-24 18:59:41.573015。现在需要把这个时间转换成更友好、易读的格式，比如“下午6点59分”或者“18:59”。同时要注意时区问题，假设这里的时间是本地时间。然后以自然的方式回复用户，可以说“现在是下午6点59分”或者“现在的时间是18:59”。确保回答简洁明了，符合用户的需求。
</think>

现在是下午6点59分。

> Finished chain.


{'input': '现在几点了？',
 'output': '<think>\n好的，用户问现在几点了，我需要调用get_current_time函数来获取当前时间。之前已经调用了这个函数，返回的时间是2025-07-24 18:59:41.573015。现在需要把这个时间转换成更友好、易读的格式，比如“下午6点59分”或者“18:59”。同时要注意时区问题，假设这里的时间是本地时间。然后以自然的方式回复用户，可以说“现在是下午6点59分”或者“现在的时间是18:59”。确保回答简洁明了，符合用户的需求。\n</think>\n\n现在是下午6点59分。'}

In [2]:
from langchain_ollama import ChatOllama
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain import hub

# 1. 定义你的工具
# 工具可以是任何函数，这里定义一个简单的搜索工具
def search_weather(city: str) -> str:
    """当你需要查询一个城市的天气时使用这个工具。"""
    if city == "北京":
        return "北京今天晴天，25摄氏度。"
    elif city == "上海":
        return "上海今天有雨，20摄氏度。"
    else:
        return f"抱歉，我没有{city}的天气信息。"

tools = [
    Tool(
        name="WeatherSearch",
        func=search_weather,
        description="用于查询指定城市的天气。",
    ),
]

# 2. 获取 ReAct 的提示模板
# LangChain Hub 提供了很多预设的优秀提示
prompt = hub.pull("hwchase17/react")

# 3. 初始化 LLM
llm = ChatOllama(model="qwen3:4b", temperature=0)

# 4. 创建 ReAct Agent
# create_react_agent 返回一个 Runnable 对象
agent = create_react_agent(llm, tools, prompt)

# 5. 创建 AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 6. 运行 Agent
print("--- 单次查询 ---")
agent_executor.invoke({"input": "今天北京的天气怎么样？"})

# print("\n--- 带有对话历史的查询 ---")
# from langchain_core.messages import AIMessage, HumanMessage
# agent_executor.invoke(
#     {
#         "input": "那上海呢？",
#         # 注意 ReAct 的 history 是字符串形式
#         "chat_history": "Human: 今天北京的天气怎么样？\nAI: 北京今天晴天，25摄氏度。",
#     }
# )

D:\javaDevelopment\program_project\Pycharm_project\LangChainNoteBook\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


--- 单次查询 ---


> Entering new AgentExecutor chain...
<think>
好的，用户问今天北京的天气怎么样。我需要用WeatherSearch工具来查。首先，确定工具的参数是城市名称，所以动作是WeatherSearch，输入北京。然后等待返回结果，根据结果给出回答。
</think>

Action: WeatherSearch  
Action Input: 北京  北京今天晴天，25摄氏度。<think>
好的，用户问今天北京的天气，我需要用WeatherSearch工具来查。输入北京，得到结果是晴天，25摄氏度。现在可以给出答案了。
Final Answer: 今天北京的天气是晴天，温度为25摄氏度。

> Finished chain.


{'input': '今天北京的天气怎么样？', 'output': '今天北京的天气是晴天，温度为25摄氏度。'}

In [2]:
# conversational_agent_example.py

import os
from langchain_ollama import ChatOllama
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# --- 1. 设置模型 ---
# 确保你的 Ollama 服务正在本地运行
# 将 model 设置为你已下载的 Ollama 模型名称，例如 "qwen3:4b"
# 如果 "qwen3:4b" 不可用，请替换为 "qwen:4b" 或其他你拥有的模型
print("正在初始化Ollama模型...")
llm = ChatOllama(model="qwen3:4b", temperature=0)
print("模型初始化完成。")

# --- 2. 定义工具 ---
# Agent 可以使用的工具列表。这里我们只提供一个简单的搜索工具。
print("正在设置工具...")
search_tool = DuckDuckGoSearchRun()
tools = [search_tool]
print("工具设置完成。")

# --- 3. 创建 Agent 的 Prompt ---
# 这是指导 Agent 如何行动的核心指令。
# 它需要包含几个关键的占位符：
# - input: 用户的当前输入
# - chat_history: 对话历史记录
# - agent_scratchpad: Agent 的思考过程和工具调用记录
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. You have access to a search tool to look up information."),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# --- 4. 创建 Agent ---
# `create_tool_calling_agent` 是一个高级函数，它将 LLM、工具和 Prompt 组合成一个可运行的 Agent。
print("正在创建Agent...")
agent = create_tool_calling_agent(llm, tools, prompt)
print("Agent创建完成。")

# --- 5. 创建 Agent 执行器 ---
# AgentExecutor 负责实际运行 Agent，调用工具，并返回结果。
# `verbose=True` 会打印出 Agent 的完整思考链，便于调试和理解。
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- 6. 为 Agent 添加记忆功能 ---
# 为了实现对话功能，我们需要一个地方来存储历史消息。
# ChatMessageHistory 是一个简单的内存存储方案。
message_history = ChatMessageHistory()

# RunnableWithMessageHistory 包装了我们的 AgentExecutor，
# 它可以自动处理从 message_history 中加载和保存消息。
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # `lambda session_id: message_history` 表示所有会话共享同一个历史记录
    # 在实际应用中，你可能会根据 session_id 为每个用户创建不同的历史记录
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
print("Agent已添加记忆功能。准备开始对话！")

# --- 7. 与 Agent 互动 ---
print("\n--------------------------------------------------")
print("第一次提问 (需要使用工具)")
# 第一次调用，Agent 需要使用搜索工具来回答问题
response1 = agent_with_chat_history.invoke(
    {"input": "LangChain 这个项目的最新版本是多少？"},
    # `configurable` 参数是 RunnableWithMessageHistory 所必需的
    config={"configurable": {"session_id": "user123"}}
)

print("\n[Agent最终回答]:")
print(response1["output"])
print("--------------------------------------------------")


print("\n--------------------------------------------------")
print("第二次提問 (測試记忆功能)")
# 第二次调用，Agent 应该能记住上一个问题，并理解 "它" 指的是 "LangChain"
response2 = agent_with_chat_history.invoke(
    {"input": "它主要是用什么编程语言开发的？"},
    config={"configurable": {"session_id": "user123"}}
)

print("\n[Agent最终回答]:")
print(response2["output"])
print("--------------------------------------------------")

# --- 8. 查看对话历史 ---
print("\n--- 查看最终对话历史 ---")
print(message_history.messages)


正在初始化Ollama模型...
模型初始化完成。
正在设置工具...
工具设置完成。
正在创建Agent...
Agent创建完成。
Agent已添加记忆功能。准备开始对话！

--------------------------------------------------
第一次提问 (需要使用工具)


> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'LangChain latest version'}`
responded: <think>
好的，用户问的是LangChain项目的最新版本是多少。我需要先确认一下当前最新的版本号。LangChain是一个用于构建语言模型应用程序的框架，可能经常更新，所以得查一下最近的发布信息。

首先，我应该使用提供的工具duckduckgo_search来搜索最新的版本。搜索关键词应该是“LangChain latest version”或者“LangChain current version”。可能需要检查官方文档或者GitHub仓库的发布页面。

然后，我需要分析搜索结果，找到最新的版本号。通常，版本号会以语句形式出现，比如“v0.23.0”或者类似的格式。可能需要查看GitHub的 releases 页面或者 pip 的包信息。

另外，要注意是否有多个版本号，比如开发版和稳定版的区别。用户可能需要的是稳定版的最新版本，所以需要确认哪个是正式发布的。

假设搜索结果中显示最新的稳定版本是v0.23.0，发布于2023年10月。那么答案就是这个版本号。如果搜索结果中有不同的版本，需要进一步验证哪个是最新。

最后，将找到的版本号整理成自然语言的回答，确保准确无误。
</think>





D:\javaDevelopment\program_project\Pycharm_project\LangChainNoteBook\.venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


4 days ago · 1、LangChain介绍 LangChain 就是一个 LLM 编程框架，你想开发一个基于 LLM 应用，需要什么组件它都有，直接使用就行；甚至针 … Jun 29, 2023 · 1、LangChain 是什么 LangChain 是一个框架，用于开发由 LLM 驱动的应用程序。可以简单认为是 LLM 领域的 Spring，以及开源版的 … Jun 29, 2023 · LangChain 通常被用作「粘合剂」，将构建 LLM 应用所需的各个模块连接在一起。使用Langchain中不同组件的特性和能力，可以构建 … Jun 1, 2023 · 从零开始学LangChain（7）：十分钟用LangChain做一个产品客服助手 上一篇文章里，借助于LangChain，我们只用了十几行代码就实现 … 主要优点是，你用自然语言提出问题，LangChain把这个问题丢给LLMs，LLMs返回SQL脚本，然后LangChain会自动执行查询语句得到结果后，再 …<think>
好的，用户问的是LangChain的最新版本。我需要先确认最新的版本号。根据之前的搜索结果，用户提供的搜索结果中并没有直接提到最新版本号。不过，从时间来看，这些信息是4天前的，可能已经过时了。

首先，我应该考虑LangChain的官方文档或者GitHub仓库的发布页面，这些地方通常会有最新的版本信息。另外，也可以查看PyPI上的包信息，因为通常最新版本会在那里更新。

假设现在的时间是2023年10月，而搜索结果中的信息是4天前的，可能最新的版本已经是v0.23.0或者更高。不过，为了准确起见，我需要再次进行搜索，确保获取最新的版本号。

可能的步骤是：使用DuckDuckGo搜索“LangChain latest version”或者“LangChain current version”，然后查看结果中的官方来源，比如GitHub的Releases页面或者官方博客。例如，如果最新版本是v0.23.0，发布于2023年10月，那么这就是答案。

另外，需要注意是否有beta版本或者开发版本，用户可能需要的是稳定版。因此，在搜索结果中需要确认哪个版本是正式发布的，并且是最新版本。

总结：根据现有信息，可能需要再次搜索确认，但根据之前的搜索结果，可能最新的稳定版本是v0.23

In [3]:

"""
快速启动版本 - 最小可行的对话式 Agent
"""

from langchain_ollama import ChatOllama
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 定义简单工具
@tool
def simple_search(query: str) -> str:
    """简单搜索功能"""
    data = {
        "python": "Python是一种编程语言",
        "ai": "人工智能技术",
        "weather": "今天天气不错"
    }
    return data.get(query.lower(), f"未找到关于{query}的信息")

@tool
def simple_calc(expression: str) -> str:
    """简单计算器"""
    try:
        result = eval(expression)
        return f"{expression} = {result}"
    except:
        return "计算错误"

# 快速创建 Agent
def create_quick_agent():
    llm = ChatOllama(model="qwen3:4b", temperature=0)
    tools = [simple_search, simple_calc]

    prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个助手，可以搜索和计算。"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}")
    ])

    agent = create_tool_calling_agent(llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    memory = ChatMessageHistory()
    return RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: memory,
        input_messages_key="input",
        history_messages_key="chat_history"
    )

# 测试
if __name__ == "__main__":
    agent = create_quick_agent()

    # 测试对话
    queries = [
        "搜索 python",
        "计算 10 + 20",
        "刚才搜索的是什么？"
    ]

    for query in queries:
        print(f"\n用户: {query}")
        result = agent.invoke(
            {"input": query},
            config={"configurable": {"session_id": "test"}}
        )
        print(f"助手: {result['output']}")


用户: 搜索 python


> Entering new AgentExecutor chain...

Invoking: `simple_search` with `{'query': 'python'}`
responded: <think>
好的，用户让我搜索“python”，我需要使用simple_search函数来处理这个请求。首先，确认一下这个函数的参数是需要一个query字段，类型是字符串。用户输入的关键词是“python”，所以应该直接调用这个函数，把查询词传进去。不需要其他参数，因为用户没有指定其他条件。接下来，我需要确保正确构造JSON对象，函数名是simple_search，参数是{"query": "python"}。这样应该就能返回用户需要的搜索结果了。没有其他需要处理的步骤，直接调用即可。
</think>



Python是一种编程语言<think>
好的，用户之前让我搜索“python”，现在返回的结果是“Python是一种编程语言”。我需要确认这个回答是否符合用户的需求。用户可能是在学习编程，或者需要了解Python的基本信息。接下来，我应该补充更多关于Python的细节，比如它的应用场景、优点或者基本语法示例，以便用户获得更全面的信息。同时，考虑到用户可能对编程不太熟悉，需要用简单明了的语言解释，避免使用专业术语过多。另外，可以询问用户是否需要更详细的信息或有其他问题，以提供进一步的帮助。确保回答既准确又易于理解，满足用户的实际需求。
</think>

Python是一种高级编程语言，以其简洁的语法和强大的功能而闻名。它广泛应用于Web开发、数据分析、人工智能、科学计算、自动化脚本等领域。Python的语法接近自然语言，使得初学者容易上手，同时也支持复杂的编程任务。如果你需要更详细的解释或示例，可以随时告诉我！

> Finished chain.
助手: <think>
好的，用户之前让我搜索“python”，现在返回的结果是“Python是一种编程语言”。我需要确认这个回答是否符合用户的需求。用户可能是在学习编程，或者需要了解Python的基本信息。接下来，我应该补充更多关于Python的细节，比如它的应用场景、优点或者基本语法示例，以便用户获得更全面的信息。同时，考虑到用户可能对编程不太熟悉，需要用简单明了的语言